# Introduction

So far we've identified triples of tables $T_1$, $T_2$, and $T_3$ that
share many of the same values.

Next, we want to know what is the specific relationship between the tables.
Relations are predicates that hold for those cells.

## Predicates / Operations

There are three types of relations/predicates that we consider:
1. unary, things like some subset of cells being sorted, or constituting a row/col,
2. binary, which can also be understood as a transformation, and
3. ternary, which are things like joins, unions, or some other merging operation.

The unary relations are:
- sorted,
- row,
- col,
- unique,
- same_type.

The binary relations we consider are:
- Rearrange/same_val cells (move to a different position).
- Subset (the values in A is a subset of the values in B).
- above/below/right/left

The ternary relations we consider are:
- join/v-/hlookup
- union(A,B,C) (which is just subset(A,C), subset(B,C), plus something to ensure that there is nothing in C that is not in A or B)
- concat

(Note there are many other relations/operations that we could add, but let us use these to illustrate the point)

Now, there are many relationships between these predicates/constraints.
For example, for tables (or subsets of tables) A, B, and C, concat(A,B,C) implies subset(A,C) and subset(B,C).
Or, join(A,B,C) implies subset(A,C) and subset(B,C).
Or, concat(A,B,C), row(A) implies row(C).

Indeed, we can define some of the predicates in terms of others with rules:
concat(A,B,C) :- subset(A,C), subset(B,C), <some other predicates to ensure the shape stays the same...>.

## Example
Let us use join as an example.
Let A = [[1],
         [5]]
and B = [[1,a],
         [3,b],
         [5,c]].
Then, the join of A and B is
C = [[1,a],
     [5,c]].
Note that join actually must have more than 3 arguments, and is therefore not truly ternary.
We have to specify the key columns in A and B, so
join(Ak,A,Bk,B,C), where col(Ak),subset(Ak,A), col(Bk), subset(Bk,B), and C is the result of the join.

## Program synthesis
Given three tables, find predicates that hold between them.
This is fairly easy, since we can just test all possibilities.
What is more difficult, is to find all predicates that hold on any subset of tables between them.

We consider the following approaches:
- Greedily choose the cells that make each predicate true, starting with the unary.
- Use a SAT solver to find the predicates that hold.
- Generate and test all combinations of predicates.

## Mining
We also want to find the predicates that hold between tables in a large corpus of tables.
Now it is even more important to find candidates quickly, since we would have to evaluate many pairs.
We definitely do not want to compare all pairs and especially all triples.
So we use the relations we have already found to prune the search space.

We can also do some planning since we know how long it takes to evaluate each predicate.

## Implementation
We implement the language in Python, using the following pattern:
We define a class for each predicate.
For example,
if a is a table, a = Table([[1,2],[3,4]]), then
row_a_constraint = Row(a) is an object with the following methods:
row_a_constraint.satisfiable() -> bool (False in this case)

We can also do
b = Table()  # No data so this is a variable
row_b_constraint = Subset(b,a) and Row(b)

Now we have to solve for b:
solve([row_b_constraint]) -> b = [[1,2],[3,4]].
so b.value = [[1,2]] is one possible solution.

(NOTE: maybe row should be binary Row(a, A), a is a row in table A)

In this way we can build up constraints and solve them.

## Defining new predicates
We can also define new predicates in terms of others.

If we want to remove rows from a table where those rows are present in another table, for example,
In prolog or ASP we would write:
```prolog
minus_rows(F,S,K,R) :- subset(S,F), col(S), notsubset(S,R), subset(R,S).
```
Where R is taken to be the maximum number of cells that makes the predicate true when
going in the forward direction.

In our implementation, we compose predicates by adding constraints.
```python
class MinusRows(Constraint):
    def __init__(self, full, subs, keys, result):
        super().__init__(full=full, subs=subs, keys=keys, result=result)
    def constraint():
        return subset(self.subs, self.full) \
            and col(self.subs) \
            and notsubset(self.subs, self.result) \
            and subset(self.result, self.subs)
```

## Reasoning
We can also reason about the predicates we have.
For example, say we want to find cases where MinusRows were used in a large set of tables.

For each of the conditions in our definition, we estimate the number of pairs for which that
condition will be true and the time it will take to evaluate it on the full sample, and
also the distribution over cells that will be found.

So, for example:
- subset(result, subs): 200s, 1000 pairs, Poisson(lambda=5)
- notsubset(subs, result): 100s, 50000 pairs, Poisson(lambda=2000)

Now we can see that the first one is more expensive to calculate, but will prune the
search space more, so we should do it first.

For each order of operations, we can estimate the total time taken, and the
number of instances we will lose because we can only store the largest cases.

## Evaluation
To evaluate the language, we define the following tasks:
- **Predicate synthesis**: Given a small set of tables, find the predicates that hold between them.
  - This is evaluated by synthetically creating a dataset from real tables.
  - The main evaluation is whether we can find the predicates that hold between them and how long it takes.
- **Predicate mining**: Given a set of tables, find the predicates that hold between them.
  - This is evaluated by reporting the running time
  - Giving descriptive statistics of the answers.

## Notes
Refinement search.
Minimal predicates:
+ Row(a): a is a row
+ Col(a): a is a column
+ Filter(a, b): b is a subset of a
+ Subset(a, b): the values in b are a subset of the values in a
- Rearrange(a, b): the values in b are the same as the values in a, but in a different position
- Transpose(a, b): b is a transpose of a
- VJoin(a, b, k, r): r is a join of a and b on the key k
- Union(a, b, r): r is a union of a and b
- VConcat(a, b, r): r is a vertical concatenation of a and b
- Unique(a): the values in a are unique
- Sorted(a): the values in a are sorted

## Prototype
1. Implement some of the predicates in Python
2. Test that they can be used as transformations
3. Test that they can be used in reverse
4. Test that they can be used for synthesis
5. Test the mining

In [4]:
import cpmpy as cp
import numpy as np

In [183]:
class Table:
    def __init__(self, data=None):
        # Data is a Nx3 matrix
        # First column is i (row number),
        # second column is j (column number),
        # third column is value (encoded as int)
        # e.g. [[0,0,1],[0,1,2],[1,0,3],[1,1,4]]
        # represents the table [[1,2],[3,4]]
        if data is not None:
            self.data = cp.cpm_array(data)
            self.ncells = data.shape[0]
        else:
            # The data is now a variable
            self.ncells = cp.intvar(0, 100, name="ncells")
            self.data = cp.intvar(0, 100, shape=(100, 3), name="data")

class Row:
    def __init__(self, table):
        self.table = table
        self.constraint = cp.all(table.data[:, 0] == table.data[0, 0])

def row(table):
    return cp.all(table.data[:, 0] == table.data[0, 0])

# rather in terms of transpose
#class Col:
#    def __init__(self, table):
#        self.table = table
#        self.constraint = cp.all(table.data[:, 1] == table.data[0, 1])

def transpose(a, b):
    #return a.data[:a.ncells, 0] == b.data[:b.ncells, 1] & a.data[:a.ncells, 1] == b.data[:b.ncells, 0]
    # Not necessarily in the same order, so use indexing
    idx = cp.intvar(0, a.ncells-1, shape=a.ncells, name="idx")
    dbv = cp.boolvar()
    return (dbv == True) & cp.all([
        (dbv & (i < a.ncells)).implies(
            a.data[i, 0] == b.data[idx[i], 1]
                & a.data[i, 1] == b.data[idx[i], 0]
                & a.data[i, 2] == b.data[idx[i], 2]
        )
        for i in range(a.ncells)
    ]) & cp.AllDifferentExceptN(idx, a.ncells) & (b.ncells == a.ncells)


def col(table):
    transposed = Table()
    return transpose(table, transposed) & row(transposed)


class Subset:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        n_a = a.data.shape[0]
        n_b = b.data.shape[0]
        idx = cp.intvar(0, n_b-1, shape=n_a, name="idx")
        cp_a = a.data[:, 2].flatten()
        cp_b = b.data[:, 2].flatten()
        dbv = cp.boolvar()
        self.constraint = (dbv == True) & cp.all([
            (b & (i < a.ncells)).implies(cp_b[idx[i]] == cp_a[i])
            for i in range(n_a)
        ])

def subset(a, b):
    # a is a subset of b
    n_a = a.data.shape[0]
    n_b = b.data.shape[0]
    idx = cp.intvar(0, n_b-1, shape=n_a, name="idx")
    cp_a = a.data[:, 2].flatten()
    cp_b = b.data[:, 2].flatten()
    dbv = cp.boolvar()
    return (dbv == True) & cp.all([
        (b & (i < n_a)).implies(cp_b[idx[i]] == cp_a[i])
        for i in range(n_a)
    ]) & cp.AllDifferentExceptN(idx, n_a) & (b.ncells > 0)

In [184]:
a = Table(np.array([[0,0,1],[0,1,2],[1,0,3],[1,1,4]]))
row1 = Row(a)
model = cp.Model([row1.constraint])
print(model.solve())

b = Table(np.array([[0,0,1],[0,1,2]]))
row2 = Row(b)
model = cp.Model([row2.constraint])
print(model.solve())

False
True


In [100]:
a = Table(np.array([[0,0,1],[0,1,2],[1,0,3],[1,1,4]]))
b = Table(np.array([[0,0,1],[1,1,2]]))
subset = Subset(a, b)
model = cp.Model([subset.constraint])
print(model.solve())
subset2 = Subset(b, a)
model = cp.Model([subset2.constraint])
print(model.solve())

False
True


In [185]:
class Filter:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        n_a = a.data.shape[0]
        n_b = b.data.shape[0]
        idx = cp.intvar(0, n_a, shape=n_b, name="idx")
        self.idx = idx
        self.constraint = cp.all([
            (i < b.ncells).implies(b.data[i, k] == a.data[idx[i], k])
            for i in range(n_b)
            for k in range(3)
        ]) & cp.Increasing(idx) & cp.AllDifferentExceptN(idx, n_a) & (b.ncells > 0)

In [186]:
a = Table(np.array([[0,0,1],[0,1,2],[1,0,3],[1,1,4]]))
b = Table()
filt = Filter(a, b)
model = cp.Model([filt.constraint])
print(model.solve())

True


In [187]:
b.data.value()[:b.ncells.value(), :]

array([[0, 0, 1]])

In [ ]:
class Rearrange:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        n_a = a.data.shape[0]
        n_b = b.data.shape[0]
        idx = cp.intvar(0, n_a-1, shape=n_b, name="idx")
        cp_a = cp.cpm_array(a.data[:, 2].flatten())
        cp_b = cp.cpm_array(b.data[:, 2].flatten())
        self.constraint = cp.all([
            (i < n_b).implies(cp_b[i] == cp_a[idx[i]])
            for i in range(n_b)
        ])